---
#**Series de tiempo Coronavirus en Sud América**
Datos obtenidos del repositorio de la [Universidad Johns Hopkins](https://github.com/CSSEGISandData/COVID-19)

[https://github.com/CSSEGISandData/COVID-19/issues/650](https://github.com/CSSEGISandData/COVID-19/issues/650)

---


**Detalles**

[Dataset] (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

*Última actualización: Jueves 19 de Marzo 2020*

---

In [0]:
#! git clone https://github.com/CSSEGISandData/COVID-19.git

**Código**

In [0]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [0]:
def plot_file(file, title, limit=50):
    df = pd.read_csv(file)
    #drop unused columns
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])

    #sum province/state from the same country
    #Paises de Sudamérica
    paises = ['Brazil', 'Argentina', 'Chile', 'Peru', 'Paraguay', 'Bolivia', 'Uruguay']
    df = df[df['Country/Region'].isin(paises)]

    #aggregate per country and transpose the table
    per_country = df.groupby('Country/Region').sum().transpose()
    #convert string date to date obj
    per_country.index = per_country.index.map(lambda x: datetime.strptime(x, '%m/%d/%y'))
    per_country.sort_index(inplace=True)
    keep_countries = per_country.columns[per_country.max() >= limit]
    per_country = per_country[keep_countries]

    #plot
    fig = go.Figure()
    for country in per_country.columns:
        fig.add_trace(go.Scatter(x=per_country.index, y=per_country[country],
                        mode='lines+markers',
                        hovertemplate = f"<b>{country}</b><br><br>" +
                                    "%{yaxis.title.text}: %{y:,.0f}<br>" +
                                    "%{xaxis.title.text}: %{x}<br>" +
                                    "<extra></extra>",
                        name=country))
    
    updatemenus = list([
        dict(active=1,
             buttons=list([
                dict(label='Escala log',
                     method='update',
                     args=[{'visible': [True] * len(per_country.columns)},
                           {'title': f'{title} (Escala log)',
                            'yaxis': {'type': 'log', 'title': 'Número de casos'}}]),
                dict(label='Escala lineal',
                     method='update',
                     args=[{'visible': [True] * len(per_country.columns)},
                           {'title': f'{title} (Escala lineal)',
                            'yaxis': {'type': 'lineal', 'title': 'Número de casos'}}])
                ]),
            )
    ])
    
    #customize layout
    fig.update_layout(title=f'{title} (Escala lineal)',
                      xaxis_title='Fecha',
                      yaxis_title='Número de casos',
                      autosize=True,
                      height=600,
                      updatemenus=updatemenus,
                      annotations = [dict(xref='paper',
                                          yref='paper',
                                          x=0.5, y=1.1,
                                          showarrow=False,
                                          text = f'Solo paises con mas de {limit} {title}<br>en cualquier momento de tiempo')])
    fig.show()


**Casos confirmados**

In [4]:
plot_file(
    'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv',
    'Casos confirmados', limit=2)

**Recuperados**

In [5]:
plot_file(
    'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv',
    'Casos Recuperados', limit=2)

**Fallecidos**

In [6]:
plot_file(
    'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv', 
     'Fallecidos', limit=2)